<a href="https://colab.research.google.com/github/mantoan-thi/-Python-for-investments-/blob/main/Stock_portfolio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade pandas-datareader

In [82]:
import pandas as pd
from pandas_datareader import data
from fbprophet import Prophet
from fbprophet.plot import plot_plotly, plot_components_plotly
import time

In [128]:
tickers = ['RRRP3.SA',	'CSAN3.SA',	'IGTI11.SA',	'PETZ3.SA','ALPA4.SA','CPFE3.SA','GNDI3.SA','POSI3.SA',
          'ABEV3.SA',	'CMIN3.SA',	'IRBR3.SA',	'QUAL3.SA','AMER3.SA',	'CVCB3.SA',	'ITSA4.SA',	'RADL3.SA',
          'ASAI3.SA',	'CYRE3.SA',	'ITUB4.SA',	'RDOR3.SA','AZUL4.SA',	'DXCO3.SA',	'JBSS3.SA',	'RAIL3.SA',
          'B3SA3.SA',	'ECOR3.SA',	'JHSF3.SA',	'SBSP3.SA','BIDI11.SA',	'ELET3.SA',	'KLBN11.SA',	'SANB11.SA',
          'BPAN4.SA',	'ELET6.SA',	'RENT3.SA',	'CSNA3.SA','BBSE3.SA',	'EMBR3.SA',	'LCAM3.SA',	'SULA11.SA',
          'BRML3.SA',	'ENBR3.SA',	'LWSA3.SA',	'SUZB3.SA','BBDC3.SA',	'ENGI11.SA',	'LAME4.SA',	'TAEE11.SA',
          'BBDC4.SA',	'ENEV3.SA',	'LREN3.SA',	'VIVT3.SA','BRAP4.SA',	'EGIE3.SA',	'MGLU3.SA',	'TIMS3.SA',
          'BBAS3.SA',	'EQTL3.SA',	'MRFG3.SA',	'TOTS3.SA','BRKM5.SA',	'EZTC3.SA',	'CASH3.SA',	'UGPA3.SA',
          'BRFS3.SA',	'FLRY3.SA',	'BEEF3.SA',	'USIM5.SA','BPAC11.SA',	'GGBR4.SA',	'MRVE3.SA',	'VALE3.SA',
          'CRFB3.SA',	'GOAU4.SA',	'MULT3.SA',	'VIIA3.SA','CCRO3.SA',	'GOLL4.SA',	'PCAR3.SA',	'VBBR3.SA',
          'CMIG4.SA',	'NTCO3.SA',	'PETR3.SA',	'WEGE3.SA','CIEL3.SA',	'SOMA3.SA',	'PETR4.SA',	'YDUQ3.SA',
          'COGN3.SA',	'HAPV3.SA',	'PRIO3.SA',	'HYPE3.SA','CPLE6.SA']

In [134]:
def coletar(tickers):
    start_date = '2015-01-01'
    end_date = '2016-12-31'
    dados = data.DataReader(tickers, 'yahoo', start_date)['Adj Close']
    dataset = pd.DataFrame(dados)
    dataset.reset_index(inplace=True)
    dataset.rename(columns={'Date': 'ds','Adj Close':'y'},inplace =True)
    return dataset

def previsao(dados):
    prior_scale=0.2
    m = Prophet(changepoint_prior_scale=prior_scale)
    m.fit(dados)
    future = m.make_future_dataframe(periods=30)
    forecast = m.predict(future)
    #forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()
    return forecast['yhat'].iloc[-31:]

In [136]:
def gerar_analise():
  lista = []
  contador = 0
  print('Carteira Teórica do IBovespa\n')
  print('Quantidade de ativos: ',len(tickers),'\n')
  for ativo in tickers:
      print('Analisando: ',ativo,' // ',contador,' de ',len(tickers),' {:.2%}'.format(contador/len(tickers)))
      dataset = previsao(coletar(ativo))
      var = dataset[-1:].values[0]/dataset[:1].values[0]-1
      lista.append([ativo,var])
      contador +=1
      time.sleep(1)
  new_lista = pd.DataFrame(lista,columns=['Ativo','Percentual'])
  Ativo_compra = new_lista[new_lista['Percentual'] > 0]
  Ativo_venda = new_lista[new_lista['Percentual'] < 0]
  Ativo_compra.sort_values(by=['Percentual'],ascending=False,inplace=True)
  Ativo_venda.sort_values(by=['Percentual'],inplace=True)
  Ativo_compra['Percentual'] = Ativo_compra['Percentual'].apply(lambda x: '{:.2%}'.format(x))
  Ativo_venda['Percentual'] = Ativo_venda['Percentual'].apply(lambda x: '{:.2%}'.format(x))
  return Ativo_compra,Ativo_venda

In [152]:
Ativo_compra, Ativo_venda = gerar_analise()

Carteira Teórica do IBovespa

Quantidade de ativos:  93 

Analisando:  RRRP3.SA  //  0  de  93  0.00%


INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Analisando:  CSAN3.SA  //  1  de  93  1.08%


INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Analisando:  IGTI11.SA  //  2  de  93  2.15%


...

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Analisando:  HYPE3.SA  //  91  de  93  97.85%


INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Analisando:  CPLE6.SA  //  92  de  93  98.92%


INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user

In [153]:
print('Compra:',len(Ativo_compra),'// Venda: ',len(Ativo_venda))

Compra: 73 // Venda:  20


In [157]:
Ativo_compra.head(10)

,Ativo,Percentual
78,PCAR3.SA,31.72%
84,CIEL3.SA,15.75%
20,AZUL4.SA,12.79%
10,IRBR3.SA,12.58%
32,BPAN4.SA,11.61%
43,SUZB3.SA,9.96%
13,CVCB3.SA,8.42%
40,BRML3.SA,7.78%
49,ENEV3.SA,7.54%
60,BRKM5.SA,7.18%


In [155]:
Ativo_venda

,Ativo,Percentual
62,CASH3.SA,-41.68%
81,NTCO3.SA,-34.54%
46,LAME4.SA,-34.28%
42,LWSA3.SA,-23.36%
19,RDOR3.SA,-20.51%
3,PETZ3.SA,-15.54%
88,COGN3.SA,-14.89%
2,IGTI11.SA,-13.69%
0,RRRP3.SA,-11.52%
85,SOMA3.SA,-11.14%
